In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback
import zipfile
import os
import gdown

In [ ]:
# Google Drive file URL
file_url = "https://drive.google.com/uc?id=1vHCvd9ZFkY9lfNwUnnMUbDQZjjpby8lv"

# Specify the output file path (the path where you want to save the file)
output_file = "disease.zip"  # Replace with actual file name

# Download the file
gdown.download(file_url, output_file, quiet=False)

zip_file = "disease.zip"  # Name of the downloaded file

# Make sure output directory exists
output_dir = "/content/extracted_file"
os.makedirs(output_dir, exist_ok=True)

# Extract the contents
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1NfuMblbAumQHfcuJAQCMFA4gR8gf0F8F
From (redirected): https://drive.google.com/uc?id=1NfuMblbAumQHfcuJAQCMFA4gR8gf0F8F&confirm=t&uuid=7d9e833d-d983-4527-b135-53479e8ef84d
To: /content/disease.zip
100%|██████████| 3.85G/3.85G [00:21<00:00, 180MB/s]


In [ ]:
# Path to main train folder that has many class folders
data_dir = "/content/extracted_file/Retinal Fundus Images - Copy/train"

# Specify the exact classes you want to include
target_classes = ['Hypertensive_Retinopathy', 'Normal_Fundus']

# Image settings
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Create generators with only selected classes
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=target_classes,
    class_mode='sparse',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=target_classes,
    class_mode='sparse',
    subset='validation',
    shuffle=False
)


# Build ResNet50-based model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Fine-tune last 30 layers of ResNet50
for layer in base_model.layers[:-45]:
    layer.trainable = False
for layer in base_model.layers[-45:]:
    layer.trainable = True

# Compile with smaller learning rate for fine-tuning
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



# Train the model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
)

Found 17375 images belonging to 2 classes.
Found 1930 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 123s 177ms/step - accuracy: 0.7544 - loss: 0.4942 - val_accuracy: 0.8477 - val_loss: 0.3256
Epoch 2/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 81s 149ms/step - accuracy: 0.8735 - loss: 0.2990 - val_accuracy: 0.7865 - val_loss: 0.4818
Epoch 3/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 81s 148ms/step - accuracy: 0.9062 - loss: 0.2337 - val_accuracy: 0.9161 - val_loss: 0.1868
Epoch 4/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 81s 149ms/step - accuracy: 0.9299 - loss: 0.1836 - val_accuracy: 0.7487 - val_loss: 0.7361
Epoch 5/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 81s 148ms/step - accuracy: 0.9406 - loss: 0.1512 - val_accuracy: 0.8870 - val_loss: 0.2539
Epoch 6/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 81s 149ms/step - accuracy: 0.9539 - loss: 0.1238 - val_accuracy: 0.8067 - val_loss: 0.5855
Epoch 7/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 81s 148ms/step - accuracy: 0.9656 - loss: 0.0915 - val_accuracy: 0.9026 - val_loss: 0.2266
Epoch 8/10
543/543 ━━━━━━━━━━━━━━━━━━━━ 81s 149ms/step - accuracy: 0.9729 - loss: 

In [ ]:
model.save("Hypertensive_Retinopathy.keras")